In [180]:
# Importamos las librerías necesarias para nuestro proyecto. 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# Más adelante en el notebook importaremos nuevamente librerías. 
# Pero con enfoque en el preprocesamiento, creación y entrenamiento del modelo.

In [181]:
# Abrimos el dataset con los juegos y mostramos el último registro.
df = pd.read_csv("without_outliers.csv")
df.tail(1)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
31831,NaN,[],Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"['Early Access', 'Adventure', 'Indie', 'Action...",NaN,http://steamcommunity.com/app/681550/reviews/?...,"['Single-player', 'Stats', 'Steam Leaderboards...",4.99,True,681550.0,NaN,Positive,NaN


In [182]:
# Creamos una máscara binaria para mostrar los valores nulos.
mask = df.isna()
# Creamos un bucle que ennumere e itere en la máscara y haga las suma de cantidad de valores nulos que tiene cada registro.
for i, n in enumerate(mask.sum(axis=1)):
    # Si el número de valores nulos es mayor a 6:
    if n > 6:
        # Eliminamos el registro.
        df.drop(i, axis= 0, inplace= True)

In [183]:
# Veamos la columnas y decidamos cuales utilizaremos para el modelo.
df.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'discount_price', 'reviews_url', 'specs', 'price',
       'early_access', 'id', 'developer', 'sentiment', 'metascore'],
      dtype='object')

In [184]:
# Elegimos las siguientes y las separamos en un nuevo dataset.
to_processing = df[[ 'price','genres', 'early_access', 'tags', 'metascore', 'specs']].reset_index(drop= True)
# Veamos los primeros registros del nuevo dataset.
to_processing.head(10)

,price,genres,early_access,tags,metascore,specs
0,4.99,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",False,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",NaN,['Single-player']
1,0.00,"['Free to Play', 'Indie', 'RPG', 'Strategy']",False,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,"['Single-player', 'Multi-player', 'Online Mult..."
2,0.00,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",False,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,"['Single-player', 'Multi-player', 'Online Mult..."
3,0.99,"['Action', 'Adventure', 'Casual']",False,"['Action', 'Adventure', 'Casual']",NaN,['Single-player']
4,2.99,[],False,"['Action', 'Indie', 'Casual', 'Sports']",NaN,"['Single-player', 'Full controller support', '..."
5,3.99,"['Action', 'Adventure', 'Simulation']",False,"['Action', 'Adventure', 'Simulation', 'FPS', '...",NaN,"['Single-player', 'Steam Achievements']"
6,9.99,"['Free to Play', 'Indie', 'Simulation', 'Sports']",False,"['Free to Play', 'Indie', 'Simulation', 'Sports']",NaN,"['Single-player', 'Multi-player', 'Online Mult..."
7,18.99,"['Free to Play', 'Indie', 'Simulation', 'Sports']",False,"['Free to Play', 'Indie', 'Simulation', 'Sports']",NaN,"['Single-player', 'Multi-player', 'Online Mult..."
8,29.99,"['Free to Play', 'Indie', 'Simulation', 'Sports']",False,"['Free to Play', 'Indie', 'Simulation', 'Sports']",NaN,"['Single-player', 'Multi-player', 'Online Mult..."
9,NaN,"['Casual', 'Indie', 'Racing', 'Simulation']",False,"['Indie', 'Casual', 'Simulation', 'Racing']",NaN,"['Single-player', 'Multi-player', 'Partial Con..."


In [185]:
# Aplicamos las transfomraciones a las columnas que poseen cadenas string, teniendo que ser, en realidad, listas.
import ast

In [186]:
# Transformamos primero, y en orden, la columna genres.
to_processing['genres'] = to_processing['genres'].apply(lambda x: ast.literal_eval(x) if type(x) is str else ["nan"])
# Le aplicamos a la columna una función lambda que devuelve una lista, si el dato está en formato string, de lo contrario le damos
# la instrucción para que devuelva un NaN.

In [187]:
# Procedemos con las siguientes columnas. 
to_processing['tags'] = to_processing['tags'].apply(lambda x: ast.literal_eval(x) if type(x) is str else ["nan"])
to_processing['specs'] = to_processing['specs'].apply(lambda x: ast.literal_eval(x) if type(x) is str else ["nan"])

In [188]:
list(to_processing.iloc[0,1:])

[['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 False,
 ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 nan,
 ['Single-player']]

In [189]:
# Hacemos otras transformaciones que utilizaremos en el entrenamiento del modelo.
to_processing['genres'] = to_processing['genres'].apply(lambda x: " | ".join(x))

to_processing['price'] = to_processing['price'].fillna(float("0.00"))

to_processing['tags'] = to_processing['tags'].apply(lambda x: " | ".join(x))
to_processing['specs'] = to_processing['specs'].apply(lambda x: " | ".join(x))

to_processing['metascore'] = to_processing['metascore'].apply(lambda x: str(x))
to_processing['metascore'] = to_processing['metascore'].str.replace("nan", "78.8")

to_processing['early_access'] = to_processing['early_access'].apply(lambda x: str(x))

In [190]:
# Declaramos los valores que utilizaremos para enternar el modelo
values = list(to_processing[['genres', 'metascore', 'tags', 'early_access']].values)
proc_df = pd.DataFrame(data= [to_processing['price'].values, values], index= ["price", 'values'])
proc_df = proc_df.T 
proc_df['values'] = proc_df['values'].apply(lambda x: " / ".join(x))

In [203]:
proc_df.to_json("processed_data.json")

In [191]:
# Importamos las librerías de scikit-learn que utilizaremos en este proyecto.
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [192]:
# Creamos el preprocesador de los datos
encoder = OneHotEncoder(handle_unknown='ignore')

In [193]:
# Entrenamos y trnasformamos los datos que separamos anteriormente.
values_encoded = encoder.fit_transform(proc_df['values'].values.reshape(-1,1))
# Obtenemos el array con los precios.
price_array = proc_df['price'].values.reshape(-1,1)

In [194]:
# Sepramos los datos ne datos de entrenamiento y de testeo
X_train, X_test, y_train, y_test = train_test_split(values_encoded, price_array, test_size= 0.2, random_state= 42)

In [195]:
# Creamos el modelo Lasso
model_Lasso = Lasso(alpha= 0.025)

In [196]:
# Lo entrenamos
model_Lasso.fit(X_train, y_train)

Lasso(alpha=0.025)

In [197]:
# Testeamos
y_pred = model_Lasso.predict(X_test)

In [198]:
# Obtenemos el error cuadrático medio
mse = mean_squared_error(y_pred, y_test)
mse

63.067728253862974

In [199]:
# Realizamos una predicción, lo codificamos con el encoder y precedimos un precio.
to_predict = np.array("Indie | 80 | True").reshape(-1,1)
to_predict_enc = encoder.transform(to_predict)
model_Lasso.predict(to_predict_enc)

array([7.66452053])

In [200]:
model_ridge = Ridge(alpha= 0.75)

In [201]:
model_ridge.fit(X_train, y_train)

Ridge(alpha=0.75)

In [202]:
y_pred = model_ridge.predict(X_test)
mse = mean_squared_error(y_pred, y_test)
mse

55.74165596952672

El resto del modelo, que participa en la api, se encuentra en el archivo **model_trained.py**

A continuación verá como el modelo Ridge se desempeña mejor en el resolver el MSE.

In [235]:
df = pd.read_json("processed_data.json")

encoder = OneHotEncoder(handle_unknown='ignore')

values_encoded = encoder.fit_transform(df['values'].values.reshape(-1,1))
price_array = df['price'].values.reshape(-1, 1)

# Sepramos los datos ne datos de entrenamiento y de testeo
X_train, X_test, y_train, y_test = train_test_split(values_encoded, price_array, test_size= 0.2, random_state= 42)

# Creamos el modelo Ridge
model_ridge = Ridge(alpha= 0.75)
# Lo entrenamos
model_ridge.fit(X_train, y_train)
# Obtenemos el error cuadrático medio
y_pred = model_ridge.predict(X_test)
mse = mean_squared_error(y_pred, y_test)
print(mse)

55.74165596952672
